# Mock making: linear power spectra and bias independant terms.

In this notebook we calculate some funtions that will later be used to calculate our mock galaxy power specrum multipoles. These are:
1. The linear power spectrum with [CLASS](https://github.com/lesgourg/class_public)
2. Bias independant terms needed to calculate galaxy multipoles with [PyBird](https://github.com/pierrexyz/pybird).

We will use a [Plank 2018](https://arxiv.org/abs/1807.06209) like $\Lambda$CDM cosmology, with; $\omega_c = 0.11933$, $\omega_b = 0.02242$, $h=0.6766$, $\ln{\left(10^{10} A_s\right)} = 3.047$, and $n_s=0.9665$.

We will compute the relevant functions at $z=[0.38,0.51,0.61]$. These redshifts match those from the [PT mock-challenge](https://www2.yukawa.kyoto-u.ac.jp/~takahiro.nishimichi/data/PTchallenge/).

In [1]:
import numpy as np
from classy import Class
import pybird
from tqdm import tqdm

As with most of the other notebooks in this repo we start by specifying the repo location.

In [2]:
path_to_repo = "/home/donaldj/Desktop/EFTicg/"

We next specify where within the repo we would like to save the functions we are calculating here.

In [3]:
save_dir = path_to_repo+"data/P18/"

Now we loop over the specified redshifts and calculate and save the desired functions.

In [4]:
# Define the k-values at which we will calculate the linear power.
# We do this outside the loop as it will be the same for all redshifts.
kk = np.logspace(-5, 0, 200)

for zpk in tqdm([0.38, 0.51, 0.61]):

    # Specify our cosmology and redshift for CLASS.
    M = Class()
    M.set({'output': 'mPk',
           'P_k_max_1/Mpc': 1.0,
           'z_max_pk': zpk})
    M.set({'ln10^{10}A_s': 3.047,
           'n_s': 0.9665,
           'h': 0.6766,
           'omega_b': 0.02242,
           'omega_cdm': 0.11933,
          })
    
    
    # Calculate the linear power spectrum.
    M.compute()
        
    # Convert to (Mpc/h)^3.
    Pk = [M.pk(ki*M.h(), zpk)*M.h()**3 for ki in kk]
    
    np.save(save_dir+"z{z}/lin/Plin_P18--z-{z}.npy".format(z=zpk), np.stack([kk, Pk]))
    
    # Define all the classes required by PyBird
    common = pybird.Common(optiresum=False)
    nonlinear = pybird.NonLinear(load=True, save=True, co=common)
    resum = pybird.Resum(co=common)
    bird = pybird.Bird(kk, Pk, M.scale_independent_growth_factor_f(zpk), z=zpk, which='all', co=common)

    # Calculate the desired functions.
    nonlinear.PsCf(bird)
    bird.setPsCfl()
    resum.Ps(bird)

    np.save(save_dir+"z{z}/P11l/P110_P18--z-{z}_optiresum-False.npy".format(z=zpk), bird.P11l[0])
    np.save(save_dir+"z{z}/P11l/P112_P18--z-{z}_optiresum-False.npy".format(z=zpk), bird.P11l[1])

    np.save(save_dir+"z{z}/Ploopl/Ploop0_P18--z-{z}_optiresum-False.npy".format(z=zpk), bird.Ploopl[0])
    np.save(save_dir+"z{z}/Ploopl/Ploop2_P18--z-{z}_optiresum-False.npy".format(z=zpk), bird.Ploopl[1])
 
    np.save(save_dir+"z{z}/Pctl/Pct0_P18--z-{z}_optiresum-False.npy".format(z=zpk), bird.Pctl[0])
    np.save(save_dir+"z{z}/Pctl/Pct2_P18--z-{z}_optiresum-False.npy".format(z=zpk), bird.Pctl[1])

100%|█████████████████████████████████████████████| 3/3 [00:08<00:00,  2.90s/it]
